In [53]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import os
from os import listdir
from os.path import isfile, join

In [54]:
df = pd.read_csv('train.csv')
df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
67909,936fd5cff1c058d39817a08f58b72cae,No finding,14,R1,NaN,NaN,NaN,NaN
67910,ca7e72954550eeb610fe22bf0244b7fa,No finding,14,R1,NaN,NaN,NaN,NaN
67911,aa17d5312a0fb4a2939436abca7f9579,No finding,14,R8,NaN,NaN,NaN,NaN
67912,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0


In [55]:
# list of image ids of pictures in train_select_600 folder
dir_list = os.listdir('train_select_600')
train_select_600_ids = [dir_list.split('.')[0] for dir_list in dir_list]

In [56]:
# dataframe with pictures from train_select_600 folder
filtered_df = df[df['image_id'].isin(train_select_600_ids)]
filtered_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
9,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0
...,...,...,...,...,...,...,...,...
67899,3b887c0550e9722c9b86b7c22ad09e11,Other lesion,9,R10,254.0,1959.0,280.0,1990.0
67903,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,10,R9,240.0,1550.0,562.0,2001.0
67906,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10,1163.0,787.0,1338.0,941.0
67907,22672ab82c290c20b86863291e25ef6c,ILD,5,R9,299.0,664.0,794.0,1508.0


In [57]:
# DATA OVERWIEV
print('Count of pictures in train_select_600 folder: ', len(train_select_600_ids))

print('Count of rown in dataframe with only train_select_600 pictures: ',len(filtered_df))

unique_pictures = filtered_df['image_id'].nunique()
print("Unique pictures in filtered dataframe:", unique_pictures)

image_id_counts = filtered_df['image_id'].value_counts()

print('Sum of counts of all images: ',image_id_counts.sum())

print('\nConclusion: Each image is represented several times in the dataframe. '
      'This means that each picture shows several diagnoses')

# count occurrences of each image_id
image_id_counts = filtered_df['image_id'].value_counts()

# list of tuples with image_id and count
image_id_occurrences = [(image_id, count) for image_id, count in image_id_counts.items()]

print('\nList of images and counts: ')
image_counts_df = pd.DataFrame(image_id_occurrences, columns=['images', 'counts'])
image_counts_df

Count of pictures in train_select_600 folder:  3569
Count of rown in dataframe with only train_select_600 pictures:  30018
Unique pictures in filtered dataframe: 3569
Sum of counts of all images:  30018

Conclusion: Each image is represented several times in the dataframe. This means that each picture shows several diagnoses

List of images and counts: 


,images,counts
0,03e6ecfa6f6fb33dfeac6ca4f9b459c9,57
1,fa109c087e46fe1ea27e48ce6d154d2f,52
2,e31be972e181987a8600a8700c1ebe88,48
3,ecf474d5d4f65d7a3e23370a68b8c6a0,46
4,3a302fbbbf3364aa1a7731b59e6b98ec,46
...,...,...
3564,b90ffdff145e509470fa3e65b59fd7fc,3
3565,daf7bc9b4bfad7cd4e9c5fc4a03a9992,3
3566,425d40a4b08ec1aaccdc02ca1db78388,3
3567,863827ae84bd3b17640ce459b0022d15,3


In [58]:
# deleting images with no coordinates

NaN_rows = filtered_df[filtered_df['x_min'].isna()]

print('Amount of pictures with no coordinates:', len(NaN_rows))
filtered_df

Amount of pictures with no coordinates: 1740


,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
9,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0
...,...,...,...,...,...,...,...,...
67899,3b887c0550e9722c9b86b7c22ad09e11,Other lesion,9,R10,254.0,1959.0,280.0,1990.0
67903,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,10,R9,240.0,1550.0,562.0,2001.0
67906,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10,1163.0,787.0,1338.0,941.0
67907,22672ab82c290c20b86863291e25ef6c,ILD,5,R9,299.0,664.0,794.0,1508.0


In [59]:
# filter rows where 'xmin' is NaN
no_coordinates = filtered_df[filtered_df['x_min'].isna()]

# get unique image IDs from these rows
no_coordinates = no_coordinates['image_id'].unique()

# print or use the list of IDs
no_coordinates

array(['764205f702d380f8a4da5cffa538f48d',
       '69f5d29210d98df18120534adb990291',
       'c113ff68c59ef8d40cd4c0bd6e021a6e', ...,
       'd0d9778108b9059fd91572f77ed96e51',
       '4cc8c20eab49e61d91a7be1efc380c1c',
       '282393b621e1c9913b5e1761cad94a0a'], dtype=object)

In [60]:
# NaN values
no_coordinates = filtered_df['x_min'].isnull()
# Drop rows where the 'x_min' column is NaN
df_cleaned = filtered_df.dropna(subset=['x_min'])

# cleaned dataframe
NaN_rows_cleaned = df_cleaned[df_cleaned['x_min'].isna()]
print('Amount of pictures with no coordinates in cleaned df:', len(NaN_rows_cleaned))

print("\nDataFrame after removing rows with NaN in 'x_min' column:")
df_cleaned

Amount of pictures with no coordinates in cleaned df: 0

DataFrame after removing rows with NaN in 'x_min' column:


,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
9,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0
...,...,...,...,...,...,...,...,...
67899,3b887c0550e9722c9b86b7c22ad09e11,Other lesion,9,R10,254.0,1959.0,280.0,1990.0
67903,b53d1dd80e99ca6bcef9d592f65d3321,Pleural effusion,10,R9,240.0,1550.0,562.0,2001.0
67906,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10,1163.0,787.0,1338.0,941.0
67907,22672ab82c290c20b86863291e25ef6c,ILD,5,R9,299.0,664.0,794.0,1508.0


In [61]:
no_coordinates

2        False
3        False
6        False
7        False
9        False
         ...  
67899    False
67903    False
67906    False
67907    False
67908    False
Name: x_min, Length: 30018, dtype: bool

In [62]:
# SORTING BY IMAGE_ID

# sort the dataframe based on the frequency of image ids in descending order
sorted_df = df_cleaned.groupby('image_id').size().reset_index(name='count').sort_values(by='count', ascending=False)

# create a list of unique image ids sorted by frequency
sorted_image_ids = sorted_df['image_id'].tolist()

# create a new dataframe with rows arranged based on the sorted_image_ids
filtered_df_2 = pd.concat([df[df['image_id'] == img_id] for img_id in sorted_image_ids])

# display the new sorted dataframe
filtered_df_2

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
757,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,R9,562.0,2045.0,629.0,2112.0
1906,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,R10,2046.0,778.0,2308.0,1111.0
2087,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,R10,1766.0,634.0,2645.0,2567.0
2489,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,R8,2239.0,1592.0,2336.0,1690.0
2610,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,R8,515.0,1732.0,589.0,1799.0
...,...,...,...,...,...,...,...,...
32357,f673d781efd8b4830ce480735fdcac8d,Aortic enlargement,0,R9,975.0,685.0,1152.0,918.0
57671,f673d781efd8b4830ce480735fdcac8d,Aortic enlargement,0,R8,959.0,690.0,1173.0,964.0
35355,c47b49c8a45e467ea86c02d53936b39b,Cardiomegaly,3,R9,1296.0,1359.0,2419.0,1773.0
49742,c47b49c8a45e467ea86c02d53936b39b,Cardiomegaly,3,R8,1303.0,1348.0,2415.0,1707.0


In [63]:
# drop rad_id because we don't need it
final = filtered_df_2.drop(['rad_id'], axis=1)
final

,image_id,class_name,class_id,x_min,y_min,x_max,y_max
757,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,562.0,2045.0,629.0,2112.0
1906,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,2046.0,778.0,2308.0,1111.0
2087,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,1766.0,634.0,2645.0,2567.0
2489,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,2239.0,1592.0,2336.0,1690.0
2610,03e6ecfa6f6fb33dfeac6ca4f9b459c9,Nodule/Mass,8,515.0,1732.0,589.0,1799.0
...,...,...,...,...,...,...,...
32357,f673d781efd8b4830ce480735fdcac8d,Aortic enlargement,0,975.0,685.0,1152.0,918.0
57671,f673d781efd8b4830ce480735fdcac8d,Aortic enlargement,0,959.0,690.0,1173.0,964.0
35355,c47b49c8a45e467ea86c02d53936b39b,Cardiomegaly,3,1296.0,1359.0,2419.0,1773.0
49742,c47b49c8a45e467ea86c02d53936b39b,Cardiomegaly,3,1303.0,1348.0,2415.0,1707.0


In [64]:
# list of classes
unique_classes_array = np.array(final['class_name'].unique())
np.set_printoptions(threshold = 20)
print(unique_classes_array)

['Nodule/Mass' 'ILD' 'Lung Opacity' 'Consolidation' 'Aortic enlargement'
 'Pleural thickening' 'Calcification' 'Pulmonary fibrosis' 'Cardiomegaly'
 'Pleural effusion' 'Other lesion' 'Infiltration' 'Atelectasis'
 'Pneumothorax']


In [65]:
# shade of grey for each class
shades_of_grey = {
  "Nodule/Mass": 255,
  "ILD": 237,
  "Lung Opacity": 219,
  "Consolidation": 201,
  "Aortic enlargement": 183,
  "Pleural thickening": 165,
  "Calcification": 147,
  "Pulmonary fibrosis": 129,
  "Cardiomegaly": 111,
  "Pleural effusion": 93,
  "Other lesion": 75,
  "Infiltration": 57,
  "Atelectasis": 39,
  "Pneumothorax": 21,
  "No finding": 0,
}
print(shades_of_grey)

{'Nodule/Mass': 255, 'ILD': 237, 'Lung Opacity': 219, 'Consolidation': 201, 'Aortic enlargement': 183, 'Pleural thickening': 165, 'Calcification': 147, 'Pulmonary fibrosis': 129, 'Cardiomegaly': 111, 'Pleural effusion': 93, 'Other lesion': 75, 'Infiltration': 57, 'Atelectasis': 39, 'Pneumothorax': 21, 'No finding': 0}


In [66]:
# class order
class_priorities = {
    "Nodule/Mass": 0,
    "ILD": 1,
    "Lung Opacity": 2,
    "Consolidation": 3,
    "Aortic enlargement": 4,
    "Pleural thickening": 5,
    "Calcification": 6,
    "Pulmonary fibrosis": 7,
    "Cardiomegaly": 8,
    "Pleural effusion": 9,
    "Other lesion": 10,
    "Infiltration": 11,
    "Atelectasis": 12,
    "Pneumothorax": 13,
}

In [67]:
#ARRAY WITH UNIQUE IDS
unique_pictures_array = np.array(final['image_id'].unique())
print(unique_pictures_array)
      
dfs = {}

for image_id in unique_pictures_array:
     # extract rows with the current unique id
    df = final[final['image_id'] == image_id]
    
    # store the dataframe in the dictionary
    dfs[image_id] = df

# can access each dataframe using its unique id
example_dataframe = dfs['03e6ecfa6f6fb33dfeac6ca4f9b459c9']
print(example_dataframe)

['03e6ecfa6f6fb33dfeac6ca4f9b459c9' 'fa109c087e46fe1ea27e48ce6d154d2f'
 'e31be972e181987a8600a8700c1ebe88' ... '6a21c27cbd42e68576a134fca3c0266e'
 'f673d781efd8b4830ce480735fdcac8d' 'c47b49c8a45e467ea86c02d53936b39b']
                               image_id   class_name  class_id   x_min  \
757    03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8   562.0   
1906   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8  2046.0   
2087   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8  1766.0   
2489   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8  2239.0   
2610   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8   515.0   
3318   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8  2030.0   
5824   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8   628.0   
6628   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8  2018.0   
9194   03e6ecfa6f6fb33dfeac6ca4f9b459c9  Nodule/Mass         8  1775.0   
12042  03e6ecfa6f6fb33dfeac6ca4f9b459c9  N

In [69]:
for x, y in shades_of_grey.items():
    window_name = 'Image'
    zeros = np.zeros((100, 100), dtype=np.uint8)
    shade = y
    grey = cv2.rectangle(zeros, (2, 2), (98, 98), shade, thickness=cv2.FILLED)

    cv2.imwrite(f"grey2/{shade}.png", grey)

In [70]:
image_folder = "train_select_600"
masks_folder = "masks_folder_2"

# creating the masks folder if it doesn't exist
if not os.path.exists(masks_folder):
    os.makedirs(masks_folder)

# iterate through each dataframe in the dfs dictionary
for image_id, dataframe in dfs.items():
    
    image_from_folder = cv2.imread(image_folder + "/"+ image_id + '.png')
    h, w, c = image_from_folder.shape

    # create a black image with the same size as the original image
    img = np.zeros((h, w), dtype=np.uint8)
    
    # sort the dataframe based on class priorities
    dataframe = dataframe.sort_values(by='class_name', key=lambda x: x.map(class_priorities))
    

    # iterate through each row in the dataframe
    for index, row in dataframe.iterrows():
        # Extract coordinates and class information
        x_min, y_min, x_max, y_max = int(row['x_min']), int(row['y_min']), int(row['x_max']), int(row['y_max'])
        class_name = row['class_name']
        shade_of_gray = shades_of_grey[class_name]

        # draw a filled rectangle
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), shade_of_gray, thickness=cv2.FILLED)

    # save the generated mask image in the masks_folder
    mask_filename = f"{masks_folder}/{image_id}.png"
    cv2.imwrite(mask_filename, img)

    print(f"Mask generated for image_id: {image_id} and saved as {mask_filename}")

Mask generated for image_id: 03e6ecfa6f6fb33dfeac6ca4f9b459c9 and saved as masks_folder_2/03e6ecfa6f6fb33dfeac6ca4f9b459c9.png
Mask generated for image_id: fa109c087e46fe1ea27e48ce6d154d2f and saved as masks_folder_2/fa109c087e46fe1ea27e48ce6d154d2f.png
Mask generated for image_id: e31be972e181987a8600a8700c1ebe88 and saved as masks_folder_2/e31be972e181987a8600a8700c1ebe88.png
Mask generated for image_id: 6d5acf3f8a973a26844d617fffe72998 and saved as masks_folder_2/6d5acf3f8a973a26844d617fffe72998.png
Mask generated for image_id: ecf474d5d4f65d7a3e23370a68b8c6a0 and saved as masks_folder_2/ecf474d5d4f65d7a3e23370a68b8c6a0.png
Mask generated for image_id: 3a302fbbbf3364aa1a7731b59e6b98ec and saved as masks_folder_2/3a302fbbbf3364aa1a7731b59e6b98ec.png
Mask generated for image_id: c699f16ba0b86f474390da9515bcad7a and saved as masks_folder_2/c699f16ba0b86f474390da9515bcad7a.png
Mask generated for image_id: d8275cd2eabf34a7f7bf22bdd838bc70 and saved as masks_folder_2/d8275cd2eabf34a7f7bf2

Mask generated for image_id: 2b658b536fb15c22f623c3d6672b64d0 and saved as masks_folder_2/2b658b536fb15c22f623c3d6672b64d0.png
Mask generated for image_id: a2d8804d3d08a2afb75dd9e6fbb53010 and saved as masks_folder_2/a2d8804d3d08a2afb75dd9e6fbb53010.png
Mask generated for image_id: 6cf0ea4eb8810156bfb48f5a7dbeffd3 and saved as masks_folder_2/6cf0ea4eb8810156bfb48f5a7dbeffd3.png
Mask generated for image_id: 5268b1ff61505e50ac05fd430cc45276 and saved as masks_folder_2/5268b1ff61505e50ac05fd430cc45276.png
Mask generated for image_id: ff924bcbd38f123aec723aa7040d7e43 and saved as masks_folder_2/ff924bcbd38f123aec723aa7040d7e43.png
Mask generated for image_id: 8649d84b0796e9deccdd4f6bd8d4e998 and saved as masks_folder_2/8649d84b0796e9deccdd4f6bd8d4e998.png
Mask generated for image_id: 8be5bfabd547e1ef91f0b0b7b4d597d4 and saved as masks_folder_2/8be5bfabd547e1ef91f0b0b7b4d597d4.png
Mask generated for image_id: 024f9140bd829c346fc91fcf4009d251 and saved as masks_folder_2/024f9140bd829c346fc91

Mask generated for image_id: 54e6184c63c75a9695d7effc17969ad0 and saved as masks_folder_2/54e6184c63c75a9695d7effc17969ad0.png
Mask generated for image_id: d1dbddfd0d130d400e81deeef783007a and saved as masks_folder_2/d1dbddfd0d130d400e81deeef783007a.png
Mask generated for image_id: f51434ef988e30a05f8b0986814d9485 and saved as masks_folder_2/f51434ef988e30a05f8b0986814d9485.png
Mask generated for image_id: d7210d8d2f377ab74e46acfb4a158e79 and saved as masks_folder_2/d7210d8d2f377ab74e46acfb4a158e79.png
Mask generated for image_id: d8bed8c7cbb164f7e8dcc5708ef41f7b and saved as masks_folder_2/d8bed8c7cbb164f7e8dcc5708ef41f7b.png
Mask generated for image_id: 91a12fbbe1ad5eb62cdf97edeb122280 and saved as masks_folder_2/91a12fbbe1ad5eb62cdf97edeb122280.png
Mask generated for image_id: 508083b00dfef2ea10fe2aebee580990 and saved as masks_folder_2/508083b00dfef2ea10fe2aebee580990.png
Mask generated for image_id: d7049d549349da79c6fce9744a52e470 and saved as masks_folder_2/d7049d549349da79c6fce

Mask generated for image_id: 734bbd50e6a2265ae0092510852c9c24 and saved as masks_folder_2/734bbd50e6a2265ae0092510852c9c24.png
Mask generated for image_id: 5d8c27abe7add51ed3ede20c7d091f16 and saved as masks_folder_2/5d8c27abe7add51ed3ede20c7d091f16.png
Mask generated for image_id: c4c2b57389bb4eac8ce685dfe8f5b965 and saved as masks_folder_2/c4c2b57389bb4eac8ce685dfe8f5b965.png
Mask generated for image_id: da0364cb98ded8ccbdea0d70fc8fa38c and saved as masks_folder_2/da0364cb98ded8ccbdea0d70fc8fa38c.png
Mask generated for image_id: e583b6c45b5622f1e714f57521a22a30 and saved as masks_folder_2/e583b6c45b5622f1e714f57521a22a30.png
Mask generated for image_id: 62071e073320c21515baab6e5c55ddf8 and saved as masks_folder_2/62071e073320c21515baab6e5c55ddf8.png
Mask generated for image_id: 5de91fab780d937e6cba46c4e807bb12 and saved as masks_folder_2/5de91fab780d937e6cba46c4e807bb12.png
Mask generated for image_id: e3c5ff1eb0d55039f3bbc3c4875543c3 and saved as masks_folder_2/e3c5ff1eb0d55039f3bbc

Mask generated for image_id: 0d30dc1e0070e7a934f39452e3ad3b83 and saved as masks_folder_2/0d30dc1e0070e7a934f39452e3ad3b83.png
Mask generated for image_id: 0e8d3736396b615c0798033f37e4a481 and saved as masks_folder_2/0e8d3736396b615c0798033f37e4a481.png
Mask generated for image_id: c413baf55bfb894ee5af02d991b452ce and saved as masks_folder_2/c413baf55bfb894ee5af02d991b452ce.png
Mask generated for image_id: 96f334aaa1d7d9cad160e710c9965615 and saved as masks_folder_2/96f334aaa1d7d9cad160e710c9965615.png
Mask generated for image_id: b48d6f9b38b698b6ac534ec7ded5c4e8 and saved as masks_folder_2/b48d6f9b38b698b6ac534ec7ded5c4e8.png
Mask generated for image_id: 48c44319fb3ad32555821ee0c74e6719 and saved as masks_folder_2/48c44319fb3ad32555821ee0c74e6719.png
Mask generated for image_id: 310a5c5df24cacd7bfc923cf0ce2f310 and saved as masks_folder_2/310a5c5df24cacd7bfc923cf0ce2f310.png
Mask generated for image_id: c47bf1efb40dc9e8eb8975abcb539ab8 and saved as masks_folder_2/c47bf1efb40dc9e8eb897

...

Mask generated for image_id: ac07b01154a498c1e5bec041f620fa96 and saved as masks_folder_2/ac07b01154a498c1e5bec041f620fa96.png
Mask generated for image_id: eb3f408d6ce7818e66b775cbed9b53b1 and saved as masks_folder_2/eb3f408d6ce7818e66b775cbed9b53b1.png
Mask generated for image_id: 2c74f4196862f688266048d27224464d and saved as masks_folder_2/2c74f4196862f688266048d27224464d.png
Mask generated for image_id: 1bb134a3fb426b7a805ad086164ed865 and saved as masks_folder_2/1bb134a3fb426b7a805ad086164ed865.png
Mask generated for image_id: f4da0959856bcdd4b91e0c6949b109e3 and saved as masks_folder_2/f4da0959856bcdd4b91e0c6949b109e3.png
Mask generated for image_id: b576400860f1e271b820f959e8b4b9b8 and saved as masks_folder_2/b576400860f1e271b820f959e8b4b9b8.png
Mask generated for image_id: 1aebe196c344ddae48ec99b142bcf4c2 and saved as masks_folder_2/1aebe196c344ddae48ec99b142bcf4c2.png
Mask generated for image_id: a5dd0a76a5253cbd34b9365c78e44110 and saved as masks_folder_2/a5dd0a76a5253cbd34b93

Mask generated for image_id: 6900482a91a538ead56b483f77bcf289 and saved as masks_folder_2/6900482a91a538ead56b483f77bcf289.png
Mask generated for image_id: d1bb581de986967b12d4ee6a064d70e8 and saved as masks_folder_2/d1bb581de986967b12d4ee6a064d70e8.png
Mask generated for image_id: d5adeb5f9d36c06c466a45acfb35d4d8 and saved as masks_folder_2/d5adeb5f9d36c06c466a45acfb35d4d8.png
Mask generated for image_id: d09d12dab1ee691049e51dfd69722a09 and saved as masks_folder_2/d09d12dab1ee691049e51dfd69722a09.png
Mask generated for image_id: d087fb5597b73624c84203ca9633a2c1 and saved as masks_folder_2/d087fb5597b73624c84203ca9633a2c1.png
Mask generated for image_id: c13533b61e97274091200c6ceab55cb7 and saved as masks_folder_2/c13533b61e97274091200c6ceab55cb7.png
Mask generated for image_id: d04f6328a4f309158aabe9818ab3e2eb and saved as masks_folder_2/d04f6328a4f309158aabe9818ab3e2eb.png
Mask generated for image_id: 70e621d1855d992b74df22c66021b25d and saved as masks_folder_2/70e621d1855d992b74df2

Mask generated for image_id: 8ef74927dbf4b7819d546c7e1f2e002e and saved as masks_folder_2/8ef74927dbf4b7819d546c7e1f2e002e.png
Mask generated for image_id: 8e3006f8c3906fc969d4aa9aeb31f586 and saved as masks_folder_2/8e3006f8c3906fc969d4aa9aeb31f586.png
Mask generated for image_id: 7c53cf2eb4f08dbc905b85f144f96b45 and saved as masks_folder_2/7c53cf2eb4f08dbc905b85f144f96b45.png
Mask generated for image_id: 4ba445edf32750faf59f06cb09f1ed93 and saved as masks_folder_2/4ba445edf32750faf59f06cb09f1ed93.png
Mask generated for image_id: 61b870f53ee51305fca5b584ffaf4a8d and saved as masks_folder_2/61b870f53ee51305fca5b584ffaf4a8d.png
Mask generated for image_id: 3bd262a356f149e12cab2c957d79fbde and saved as masks_folder_2/3bd262a356f149e12cab2c957d79fbde.png
Mask generated for image_id: 420e72f8ebc531f2eaed5ee48a3203b4 and saved as masks_folder_2/420e72f8ebc531f2eaed5ee48a3203b4.png
Mask generated for image_id: 41d642577ada2969ece3637f6c800139 and saved as masks_folder_2/41d642577ada2969ece36

Mask generated for image_id: 0187da5fd71c7a2917346bf12a67bbe4 and saved as masks_folder_2/0187da5fd71c7a2917346bf12a67bbe4.png
Mask generated for image_id: 0c5ef740a9befbf439439f0b8e652a0c and saved as masks_folder_2/0c5ef740a9befbf439439f0b8e652a0c.png
Mask generated for image_id: 75ffb5f8839bef4d8719d46c29c46064 and saved as masks_folder_2/75ffb5f8839bef4d8719d46c29c46064.png
Mask generated for image_id: 7520d54e60d31f2310d1eb5c77a7507a and saved as masks_folder_2/7520d54e60d31f2310d1eb5c77a7507a.png
Mask generated for image_id: 9120a0510a95b14a2554cb6cd5319f00 and saved as masks_folder_2/9120a0510a95b14a2554cb6cd5319f00.png
Mask generated for image_id: b17fa1439061ffd2a0b36e8acd83bc32 and saved as masks_folder_2/b17fa1439061ffd2a0b36e8acd83bc32.png
Mask generated for image_id: 85d5ef7e1090859bf9594d55460d2026 and saved as masks_folder_2/85d5ef7e1090859bf9594d55460d2026.png
Mask generated for image_id: a33cd2473556d2cdfd77ba3d2fa78349 and saved as masks_folder_2/a33cd2473556d2cdfd77b

Mask generated for image_id: 52979eadb9adb0c1dc9a64fa0d3c4d26 and saved as masks_folder_2/52979eadb9adb0c1dc9a64fa0d3c4d26.png
Mask generated for image_id: f2ea94fbd14e7946114152b0a11d6ade and saved as masks_folder_2/f2ea94fbd14e7946114152b0a11d6ade.png
Mask generated for image_id: da648f4e3bf110e8c31b0fd1859c19c2 and saved as masks_folder_2/da648f4e3bf110e8c31b0fd1859c19c2.png
Mask generated for image_id: 1c42e4e14355bfa790794d44e3a1a4d2 and saved as masks_folder_2/1c42e4e14355bfa790794d44e3a1a4d2.png
Mask generated for image_id: 466b5b5f21f766aecb22148002ff865a and saved as masks_folder_2/466b5b5f21f766aecb22148002ff865a.png
Mask generated for image_id: 52951d7de2485aba8ed62629eee4d254 and saved as masks_folder_2/52951d7de2485aba8ed62629eee4d254.png
Mask generated for image_id: 1c1a57cb8b3893348ecfb6e704a1b87e and saved as masks_folder_2/1c1a57cb8b3893348ecfb6e704a1b87e.png
Mask generated for image_id: 96734f3235d72a018aaef6b78391df7a and saved as masks_folder_2/96734f3235d72a018aaef